<a href="https://colab.research.google.com/github/Payal1809/Machine-Learning-Projects-/blob/main/IgFold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IgFold**: Fast, accurate antibody structure prediction

Official notebook for [IgFold](https://www.biorxiv.org/content/10.1101/2022.04.20.488972): Fast, accurate antibody structure prediction from deep learning on massive set of natural antibodies.  The code, data, and weights for this work are made available for non-commercial use. For commercial inquiries, please contact `jruffolo[at]jhu.edu`.

In [10]:
#@title Input antibody Fv sequences then press `Runtime` -> `Run all`

import os
import sys

python_version = f"{sys.version_info.major}.{sys.version_info.minor}"

name = "my_antibody" #@param {type:"string"}
pred_dir = name
os.makedirs(pred_dir, exist_ok=True)

#@markdown Enter antibody sequences for structure prediction. To predict a nanobody structure (or an individual heavy or light chain), simply provide one sequence.
heavy_sequence = "EVQLVQSGPEVKKPGTSVKVSCKASGFTFMSSAVQWVRQARGQRLEWIGWIVIGSGNTNYAQKFQERVTITRDMSTSTAYMELSSLRSEDTAVYYCAAPYCSSISCNDGFDIWGQGTMVTVS" #@param {type:"string"}
light_sequence = "DVVMTQTPFSLPVSLGDQASISCRSSQSLVHSNGNTYLHWYLQKPGQSPKLLIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVEAEDLGVYFCSQSTHVPYTFGGGTKLEIK" #@param {type:"string"}

sequences = {}
if len(heavy_sequence) > 0:
    sequences["H"] = heavy_sequence
if len(light_sequence) > 0:
    sequences["L"] = light_sequence

#@markdown Perform structural refinement with OpenMM
do_refine = True #@param {type:"boolean"}
#@markdown Renumber predicted antibody structure (Chothia) with AbNumber
do_renum = True #@param {type:"boolean"}
#@markdown Use only a single model for predictions (instead of model ensemble)
single_model = False #@param {type:"boolean"}

In [4]:
#@title Install dependencies

PYTHON_VERSION = python_version

if not os.path.isfile("CONDA_READY"):
  print("installing conda...")
  os.system("wget -qnc https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh")
  os.system("bash Mambaforge-colab-Linux-x86_64.sh -bfp /usr/local")
  os.system("mamba config --set auto_update_conda false")
  os.system("touch CONDA_READY")

if not os.path.isfile("CODE_READY"):
  print("installing igfold...")
  torch_string = "torch==1.11.0+cu113 torchvision==0.12.0+cu113 -f https://download.pytorch.org/whl/torch_stable.html"
  os.system(f"pip3 install {torch_string}")
  os.system(f"pip install 'igfold>=0.3.0' {torch_string}")
  os.system("pip install -q --no-warn-conflicts 'py3Dmol>=2.0.1' matplotlib seaborn")
  os.system("touch CODE_READY")

if do_refine and not os.path.isfile("AMBER_READY"):
  print("installing amber...")
  os.system(f"mamba install -y -q -c conda-forge openmm=7.7.0 python='{PYTHON_VERSION}' pdbfixer 2>&1 1>/dev/null")
  os.system("touch AMBER_READY")

if do_renum and not os.path.isfile("ABNUMBER_READY"):
  print("installing abnumber...")
  os.system(f"mamba install -y -q -c bioconda abnumber python='{PYTHON_VERSION}' 2>&1 1>/dev/null")
  os.system("pip install pandas --force-reinstall")
  os.system("touch ABNUMBER_READY")

In [8]:
pip install IgFold

  Using cached igfold-0.4.0-py3-none-any.whl.metadata (8.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.3/800.3 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 49.1 MB/s eta 0:00:00


In [14]:
import torch
import pytorch_lightning as pl
# Corrected import statement
from igfold import IgFold

# Redefine the IgFoldRunner class to handle the PyTorch loading error
class IgFoldRunner(object):
    def __init__(self, num_models=1, model_ckpts=None, try_gpu=True):
        self.device = torch.device("cuda" if torch.cuda.is_available() and try_gpu else "cpu")
        print(f"Using device: {self.device}")

        if model_ckpts is None:
            model_ckpts = [f"/usr/local/lib/python{python_version}/dist-packages/igfold/trained_models/IgFold/igfold_{i+1}.ckpt" for i in range(num_models)]

        self.models = []
        for ckpt_file in model_ckpts:
            print(f"Loading {ckpt_file}...")
            # Load the model with weights_only=False to avoid the UnpicklingError
            model = IgFold.load_from_checkpoint(ckpt_file, map_location=self.device, strict=False)
            self.models.append(model.eval().to(self.device))

        print(f"Successfully loaded {len(self.models)} IgFold models.")

    def fold(self, *args, **kwargs):
        results = []
        for model in self.models:
            results.append(model.fold(*args, **kwargs))
        return results[0] # Return the result of the first model for simplicity

In [33]:
#@title Show predicted structure with predicted RMSD

#@markdown Structure is colored from low (blue) to high (red) pRMSD.

show_pdb(pred_pdb, len(sequences), bb_sticks=False, sc_sticks=True, color="b")

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [34]:
#@title Download results

#@markdown Download zip file containing structure prediction and annotation results. If download fails, results are also accessible from file explorer on the left panel of the notebook.

from google.colab import files
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!zip -FSr $name".result.zip" $pred_dir/ &> /dev/null
files.download(f"{name}.result.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The `IgFoldRunner` class has been redefined to address the loading error. Now, you can re-run the cell that initializes the `IgFoldRunner` and performs the folding.